<font color="green">*To start working on this notebook, or any other notebook that we will use in the Moringa Data Science Course, we will need to save our own copy of it. We can do this by clicking File > Save a Copy in Drive. We will then be able to make edits to our own copy of this notebook.*</font>

# SQL Programming - Subqueries using Select Statements

## 1.0 Connecting to our Database

In [ ]:
# We will first load the sql extension into our environment
%load_ext sql

# Then connect to our in memory sqlite database
# NB: This database will cease to exist as soon as the database connection is closed. 
%sql sqlite://

## 1.1 Loading Data from CSV Files

Datasets used in this notebook:


1.   **World Countries**

> ([Dataset Download Link](https://drive.google.com/a/moringaschool.com/file/d/1XlabneVWhvbmpRUi_Wix2FR5uYAAp-ZY/view?usp=sharing))

2.   **World Cities** 

> ([Dataset Download Link](https://drive.google.com/a/moringaschool.com/file/d/1R5952m-8hH91uiwDK8dcL9sT2D_PEcuw/view?usp=sharing))






In [ ]:
%load_ext sql
%sql sqlite://

'Connected: @None'

In [ ]:
# Importing Pandas
import pandas as pd
import csv 

In [ ]:

with open('world_cities.csv','r') as f:
    cities = pd.read_csv(f, index_col=0, encoding='utf-8') 

%sql DROP TABLE if EXISTS cities;
%sql  PERSIST cities;

 * sqlite://
Done.
 * sqlite://


'Persisted cities'

In [ ]:
# Loading our cities dataset 
# 
with open('world_countries.csv','r') as f:
     countries = pd.read_csv(f, index_col=0, encoding='utf-8') 

%sql DROP TABLE if EXISTS countries;
%sql PERSIST countries;

 * sqlite://
Done.
 * sqlite://


'Persisted countries'

## 1.2 Subqueries using the SELECT statement

A subquery is a query within another query. The outer query is called as main query and inner query is called as subquery. The subquery must be enclosed with parenthesis and generally executes first, and its output is used to complete the query condition for the main or outer query. We will go through examples of how subqueries can be used to answers different types of questions then later, we will use our learnings to work on the challenges.

In [ ]:
%%sql
SELECT * FROM cities LIMIT 5

 * sqlite://
Done.


city,country,latitude,longitude,temperature
Aalborg,Denmark,57.03,9.92,7.52
Aberdeen,United Kingdom,57.17,-2.08,8.1
Abisko,Sweden,63.35,18.83,0.2
Adana,Turkey,36.99,35.32,18.67
Albacete,Spain,39.0,-1.87,12.62


In [ ]:
%%sql
SELECT* FROM countries  LIMIT 5

 * sqlite://
Done.


country,population,EU,coastline
Albania,2.9,no,yes
Andorra,0.07,no,no
Austria,8.57,yes,no
Belarus,9.48,no,no
Belgium,11.37,yes,yes


In [ ]:
# Example 1
# Subqueries can occur in the select list of the containing query
# as shown below;
# Let's find all countries in the Countries table with no city in the Cities table
# 
%%sql
SELECT country
FROM countries
WHERE NOT EXISTS (SELECT * FROM cities
                  WHERE cities.country = countries.country) 


 * sqlite://
Done.


country
Cyprus
Iceland
Kosovo
Liechtenstein
Luxembourg


In [ ]:
# Example 2
# Find the European countries with cities experiencing temperature > 20?
# 
%%sql
SELECT country
FROM countries
WHERE EU = 'yes'
AND EXISTS (SELECT * FROM cities
            WHERE cities.country = countries.country
            AND temperature > 15)

 * sqlite://
Done.


country
Greece
Italy
Portugal
Spain


In [ ]:
# Example 3: Method 1
# Find the westernmost city, returning the city and longitude
# 
%%sql
SELECT city, longitude
FROM worldcities C1
WHERE NOT EXISTS (SELECT * FROM worldcities C2
                  WHERE C2.longitude < C1.longitude)

 * sqlite://
Done.


city,longitude
Lisbon,-9.14


In [ ]:
# Example 3: Method 2
# Find the westernmost city, returning the city and longitude
# 
%%sql
SELECT city, longitude
FROM  worldcities
WHERE longitude = (SELECT min(longitude) FROM worldcities)

 * sqlite://
Done.


city,longitude
Lisbon,-9.14


In [ ]:
# Example 4:
# How many cities are in the EU?
# 
%%sql
SELECT count()
FROM cities
WHERE country IN (SELECT country FROM countries WHERE EU = 'yes')

 * sqlite://
Done.


count()
150


### <font color="green">1.2 Challenges</font>

In [ ]:
# Challenge 1
# Find number of countries that have a city with latitude > 1000 (start with country list)
# 
%%sql
SELECT count(country) FROM countries
WHERE  EXISTS (SELECT *  FROM cities WHERE  cities.country = countries.country AND latitude>50 )

 * sqlite://
Done.


count(country)
17


In [ ]:
# Challenge 2
# Find three western most cities returning both the cities and their longitude
#
%%sql
SELECT city, longitude FROM cities
WHERE longitude = (SELECT min(longitude) FROM cities)

 * sqlite://
Done.


city,longitude
Lisbon,-9.14


In [ ]:
# Challenge 3
# Find cities with temperature is more than 50% higher than the average; 
# We will return the city, country, and temperature, ordered by descending temperature
# Hint: avg(temperature) * 1.5
# 
%%sql
SELECT city, country, temperature FROM cities
WHERE temperature = (SELECT temperature > avg(temperature)*1.5 FROM cities)

 * sqlite://
Done.


city,country,temperature


In [ ]:
# Challenge 4
# How many cities are not in the EU and without a coastline?
# Hint: "not in"
# 
%%sql
SELECT count(city) FROM cities
WHERE country IN  (SELECT *  FROM countries WHERE countries.country = cities.country  AND EU = 'NO'  AND  coastline ='NO' )

 * sqlite://
(sqlite3.OperationalError) sub-select returns 4 columns - expected 1
[SQL: SELECT count(city) FROM cities
WHERE country IN  (SELECT *  FROM countries WHERE countries.country = cities.country  AND EU = 'NO'  AND  coastline ='NO' )]
(Background on this error at: http://sqlalche.me/e/14/e3q8)


In [ ]:
# Challenge 5
# Find the number of countries with no coastline and a city with longitude < 10
# 
%%sql
SELECT count(country) FROM countries 
WHERE coastline = 'NO' 
AND EXISTS (SELECT *  FROM cities WHERE countries.country = cities.country AND longitude >10)

 * sqlite://
Done.


count(country)
0


In [ ]:
# Challenge 6
# Find countries in Countries table with no city in Cities table
#

%%sql
SELECT country FROM Countries
WHERE NOT EXISTS (SELECT * FROM Cities  WHERE countries.country = cities.country )

 * sqlite://
Done.


country
Cyprus
Iceland
Kosovo
Liechtenstein
Luxembourg


In [ ]:
# Challenge 7
# Find all cities in a country whose population is < 2, returning the city and country.
# 
%%sql
SELECT  city  FROM cities
WHERE EXISTS (SELECT * FROM countries WHERE countries.country = cities.country AND population >2)

 * sqlite://
Done.


city
Aalborg
Aberdeen
Abisko
Adana
Albacete
Algeciras
Amiens
Amsterdam
Ancona
Angers


In [ ]:
# Challenge 8
# Find all countries with no city having a temperature > 6
# 
%%sql
 SELECT country FROM countries
 WHERE NOT EXISTS (SELECT * FROM cities  WHERE cities.country = countries.country AND temperature > 6)

 * sqlite://
Done.


country
Cyprus
Estonia
Finland
Iceland
Kosovo
Latvia
Liechtenstein
Luxembourg
Norway
